<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/HaruhiZero%E5%AF%B9%E6%8E%A5Qwen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [ ] cutlen 2048的适配（以qwen的tokenizer为标准）
- [ ] 超长message适当切分
- [ ] 看一看source字段对llama factory有没有影响

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls /content/drive/MyDrive/CardBuild/HaruhiZero/merge_all_105k.jsonl

/content/drive/MyDrive/CardBuild/HaruhiZero/merge_all_105k.jsonl


In [3]:
input_name = "/content/drive/MyDrive/CardBuild/HaruhiZero/merge_all_105k.jsonl"
datas = []
import json
with open(input_name, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        datas.append(data)

检查一下source

In [4]:
all_source = set()

for data in datas:
    all_source.add(data["source"])

print(all_source)

{'PIPPA', 'Haruhi52K', 'Haruhi_Other_Chinese', 'Claude_Baize', 'Janitor', 'RoleLLM', 'Haruhi_Waifu_extended', 'Direct_Role_Play_Claude'}


In [5]:
!pip install -q tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-1_8B-Chat", trust_remote_code=True)


tokenizer_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenization_qwen.py:   0%|          | 0.00/9.62k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-1_8B-Chat:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


qwen.tiktoken:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

In [7]:
sample_text = "一句句子。"

def tokenizer_len( text ):
    return len(tokenizer(text)["input_ids"])
print(tokenizer_len(sample_text))

3


In [11]:
def get_sum_token( data ):
    if "system" in data:
        sum_token = tokenizer_len( data["system"] )
    else:
        sum_token = 0
    for conversation in data["conversations"]:
        sum_token += tokenizer_len( conversation["value"] )
        if sum_token > 4096:
            return sum_token
    return sum_token

print(get_sum_token(datas[0]))

735


In [18]:
cutted_source = ['Direct_Role_Play_Claude', 'PIPPA', 'Janitor', 'Claude_Baize']

new_datas = []

max_len = 1024*3

from tqdm import tqdm
import copy
error_count = 0
count = 0
for id,data in tqdm(enumerate(datas)):
    if data["source"] not in cutted_source:
        new_datas.append(data)
        continue
    if get_sum_token(data) < max_len:
        new_datas.append(data)
        continue

    if id % 1000 == 0:
        print("deal ", id, " len save = ", len(new_datas))

    if data["system"].strip() != "":
        system_prompt = data["system"]
        sys_len = tokenizer_len( data["system"] )
        msg_len = sys_len

        current_conversations = []

        conversations = data["conversations"]

        conv_len = len(conversations)

        break_id = -1

        for i in range(0, conv_len, 2):
            current_conversations += conversations[i:i+2]
            current_qa_len = tokenizer_len( conversations[i]["value"] ) + tokenizer_len( conversations[i+1]["value"] )

            msg_len += current_qa_len
            if msg_len > max_len:
                break_id = i
                break

        save_conversations = copy.deepcopy(current_conversations)
        new_data = { "system": system_prompt,
            "conversations": save_conversations,
            "source": data["source"]
        }
        new_datas.append(new_data)

        if break_id > 0 and break_id < conv_len:
            rest_conversation = conversations[break_id+2:]

            #这里给一次机会，如果rest_conversation + sys_len < max_len 则把system加上
            rest_token = get_sum_token({"conversations": rest_conversation})
            if rest_token + sys_len < max_len:
                new_data = { "system": system_prompt,
                    "conversations": rest_conversation,
                    "source": data["source"]
                }
                new_datas.append(new_data)
                continue
            else:
                convs = []
                current_len = 0
                for i in range(0, len(rest_conversation),2):
                    convs += rest_conversation[i:i+2]
                    current_qa_len = tokenizer_len( rest_conversation[i]["value"] ) + tokenizer_len( rest_conversation[i+1]["value"] )
                    current_len += current_qa_len
                    if current_len > max_len:
                        save_conversations = copy.deepcopy(convs)
                        new_data = {
                            "system": "",
                            "conversations": save_conversations,
                            "source": data["source"]
                        }
                        new_datas.append(new_data)
                        convs = []
                        current_len = 0
                if len(convs) > 0:
                    save_conversations = copy.deepcopy(convs)
                    new_data = {
                        "system": "",
                        "conversations": save_conversations,
                        "source": data["source"]
                    }
                    new_datas.append(new_data)
                continue

    else:
        #不然的话system为0
        conversations = data["conversations"]
        head_len = tokenizer_len( conversations[0]["value"] ) + tokenizer_len( conversations[1]["value"] )

        if head_len > max_len*0.5:
            new_datas.append(data)
            continue

        current_conversations = copy.deepcopy( conversations[0:2] )
        msg_len = head_len

        break_id = -1

        for i in range(2, len(conversations), 2):
            current_conversations += conversations[i:i+2]
            current_qa_len = tokenizer_len( conversations[i]["value"] ) + tokenizer_len( conversations[i+1]["value"] )
            msg_len += current_qa_len
            if msg_len > max_len:
                save_conversations = copy.deepcopy(current_conversations)
                new_data = {
                    "system": "",
                    "conversations": save_conversations,
                    "source": data["source"]
                }
                new_datas.append(new_data)
                break_id = i
                break

        if break_id > 0 and break_id < len(conversations):
            rest_conversation = conversations[break_id+2:]
            rest_token = get_sum_token({"conversations": rest_conversation})
            if rest_token + head_len < max_len:
                save_conversations = copy.deepcopy(rest_conversation)
                save_conversations = conversations[0:2] + rest_conversation
                new_data = {
                    "system": "",
                    "conversations": save_conversations,
                    "source": data["source"]
                }
                new_datas.append(new_data)
                continue
            else:
                convs = []
                current_len = 0
                for i in range(0, len(rest_conversation),2):
                    convs += rest_conversation[i:i+2]
                    current_qa_len = tokenizer_len( rest_conversation[i]["value"] ) + tokenizer_len( rest_conversation[i+1]["value"] )
                    current_len += current_qa_len
                    if current_len > max_len:
                        save_conversations = copy.deepcopy(convs)
                        new_data = {
                            "system": "",
                            "conversations": save_conversations,
                            "source": data["source"]
                        }
                        new_datas.append(new_data)
                        convs = []
                        current_len = 0
                if len(convs) > 0:
                    save_conversations = copy.deepcopy(convs)
                    new_data = {
                        "system": "",
                        "conversations": save_conversations,
                        "source": data["source"]
                    }
                    new_datas.append(new_data)
                    continue





2014it [00:18, 94.38it/s] 

deal  2000  len save =  2746


4004it [01:02, 41.83it/s]

deal  4000  len save =  6995


5003it [01:27, 31.73it/s]

...

73509it [03:47, 157.22it/s]

deal  74000  len save =  83097


85009it [06:43, 84.98it/s]

deal  85000  len save =  101236


105168it [06:44, 260.24it/s]


这里我们只有 Direct_Role_Play_Claude、PIPPA、 Janitor、Claude_Baize做超长切分处理

In [32]:
cutted_source = ['Direct_Role_Play_Claude', 'PIPPA', 'Janitor', 'Claude_Baize']

new_datas = []

max_len = 1024*3

from tqdm import tqdm
import copy
error_count = 0
for id,data in tqdm(enumerate(datas)):
    if data["source"] not in cutted_source:
        new_datas.append(data)
        continue
    if get_sum_token(data) < max_len:
        new_datas.append(data)
        continue

    if id % 1000 == 0:
        print("deal ", id, " len save = ", len(new_datas))

    if data["system"].strip() != "":
        system_prompt = data["system"]
        sys_len = tokenizer_len( data["system"] )
        msg_len = sys_len

        if msg_len > max_len*0.5:
            # 太长的system prompt
            new_datas.append(data)
            continue

        current_conversations = []

        conversations = data["conversations"]
        conv_len = len(conversations)

        for i in range(0, conv_len, 2):
            current_conversations += conversations[i:i+2]
            current_qa_len = tokenizer_len( conversations[i]["value"] ) + tokenizer_len( conversations[i+1]["value"] )

            msg_len += current_qa_len

            if msg_len > max_len:
                # 生成一次数据
                save_conversations = copy.deepcopy(current_conversations)
                new_data = { "system": system_prompt,
                    "conversations": save_conversations,
                    "source": data["source"]
                }
                new_datas.append(new_data)
                msg_len = current_qa_len + sys_len
                current_conversations = conversations[i:i+2]
                if msg_len > max_len:
                    error_count += 1

        if len(current_conversations) > 2 or conv_len == 2:
            save_conversations = copy.deepcopy(current_conversations)
            new_data = { "system": system_prompt,
                "conversations": save_conversations,
                "source": data["source"]
            }
            new_datas.append(new_data)

        continue
    else:
        #不然的话system为0
        conversations = data["conversations"]

        head_len = tokenizer_len( conversations[0]["value"] ) + tokenizer_len( conversations[1]["value"] )

        if head_len > max_len*0.5:
            new_datas.append(data)
            continue

        current_conversations = copy.deepcopy( conversations[0:2] )
        msg_len = head_len

        for i in range(2, len(conversations), 2):
            current_conversations += conversations[i:i+2]
            current_qa_len = tokenizer_len( conversations[i]["value"] ) + tokenizer_len( conversations[i+1]["value"] )
            msg_len += current_qa_len
            if msg_len > max_len:
                save_conversations = copy.deepcopy(current_conversations)

                new_data = { "conversations": save_conversations,
                    "source": data["source"],
                    "system": ""
                }
                new_datas.append(data)

                msg_len = head_len
                current_conversations = copy.deepcopy( conversations[0:2] )

        if len(current_conversations) > 2 or len(conversations) == 2:
            save_conversations = copy.deepcopy(current_conversations)
            new_data = { "conversations": save_conversations,
                "source": data["source"],
                "system": ""
            }
            new_datas.append(new_data)


2008it [00:25, 93.25it/s] 

deal  2000  len save =  2818


4005it [01:22, 30.57it/s]

deal  4000  len save =  7688


5002it [01:58, 24.27it/s]

...

72978it [04:40, 91.40it/s]

deal  74000  len save =  86074


85013it [08:37, 83.90it/s]

deal  85000  len save =  105855


105168it [08:38, 202.77it/s]


In [19]:
print(error_count)

0


In [21]:
print(len(datas))
print(len(new_datas))

105168
121431


In [24]:
for data in new_datas:
    if "system" not in data:
        print("system not in data")
    elif "conversations" not in data:
        print("conversations not in data")
    elif len(data["conversations"]) % 2!=0:
        print("conversations not even")
    elif len(data["conversations"]) == 0:
        print("conversations is empty")
        continue
    else:
        text = data["conversations"][0]
        continue
    break

conversations is empty


In [35]:
print(new_datas[500].keys())

dict_keys(['conversations', 'source', 'system'])


In [26]:
save_name = "/content/drive/MyDrive/CardBuild/HaruhiZero/merge_cut3k_121k_no_source.jsonl"

with open(save_name, 'w', encoding='utf-8') as f:
    for data in tqdm(new_datas):
        if len(data["conversations"]) == 0:
            continue
        if "source" in data:
            del data["source"]
        f.write(json.dumps(data, ensure_ascii=False) + '\n')

100%|██████████| 121431/121431 [00:23<00:00, 5101.12it/s]


---

下面来看训练

In [1]:
!pip install --upgrade huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [2]:
!git clone https://github.com/hiyouga/LLaMA-Factory.git

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 6754, done.
remote: Counting objects: 100% (773/773), done.
remote: Compressing objects: 100% (318/318), done.
remote: Total 6754 (delta 507), reused 668 (delta 453), pack-reused 5981
Receiving objects: 100% (6754/6754), 204.59 MiB | 21.96 MiB/s, done.
Resolving deltas: 100% (4853/4853), done.
Updating files: 100% (140/140), done.


In [3]:
%cd LLaMA-Factory
!pip install -q -r requirements.txt
!pip install -q einops

/content/LLaMA-Factory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00


In [4]:
!mkdir /content/dataset

In [5]:
!ls /content/LLaMA-Factory/dataset

ls: cannot access '/content/LLaMA-Factory/dataset': No such file or directory


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!ls /content/drive/MyDrive/CardBuild/HaruhiZero/*

/content/drive/MyDrive/CardBuild/HaruhiZero/chinese_dialogue_clean.jsonl
/content/drive/MyDrive/CardBuild/HaruhiZero/Chinese_erotic.jsonl
/content/drive/MyDrive/CardBuild/HaruhiZero/Chinese_NovelWriting.jsonl
/content/drive/MyDrive/CardBuild/HaruhiZero/Claude_10k.jsonl
/content/drive/MyDrive/CardBuild/HaruhiZero/Claude_7379.jsonl
/content/drive/MyDrive/CardBuild/HaruhiZero/dialogue_extract.jsonl
/content/drive/MyDrive/CardBuild/HaruhiZero/dialogue_negative.jsonl
/content/drive/MyDrive/CardBuild/HaruhiZero/Direct_Role_Play_Claude.jsonl
/content/drive/MyDrive/CardBuild/HaruhiZero/english_dialogue_clean.jsonl
/content/drive/MyDrive/CardBuild/HaruhiZero/english_dialogue_extract.jsonl
/content/drive/MyDrive/CardBuild/HaruhiZero/Haruhi52K.jsonl
/content/drive/MyDrive/CardBuild/HaruhiZero/Haruhi_Other_Chinese.jsonl
/content/drive/MyDrive/CardBuild/HaruhiZero/Haruhi_Waifu_extended.jsonl
/content/drive/MyDrive/CardBuild/HaruhiZero/Janitor.jsonl
/content/drive/MyDrive/CardBuild/HaruhiZero/merge_

input_path = "/content/drive/MyDrive/CardBuild/HaruhiZero/"
file_names = ["Chinese_erotic.jsonl","Chinese_NovelWriting.jsonl","merge_all_126k_no_source.jsonl"]

target_path = "/content/LLaMA-Factory/data/"

请为我实现一段python代码，把input_path下的几个file_names对应的jsonl文件
拷贝到target_path

然后以json格式写入target_file

In [9]:
import shutil
import os

def copy_files(input_path, file_names, target_path):
    # 确保目标路径存在
    if not os.path.exists(target_path):
        os.makedirs(target_path)

    # 遍历文件列表并拷贝每个文件
    for file_name in file_names:
        source_file = os.path.join(input_path, file_name)
        destination_file = os.path.join(target_path, file_name)

        # 检查源文件是否存在
        if os.path.exists(source_file):
            # 拷贝文件
            shutil.copy(source_file, destination_file)
            print(f"File '{file_name}' copied to {target_path}")
        else:
            print(f"File '{file_name}' not found in {input_path}")

# 要拷贝的文件路径和文件名
input_path = "/content/drive/MyDrive/CardBuild/HaruhiZero/"
file_names = ["Chinese_erotic.jsonl", "Chinese_NovelWriting.jsonl", "merge_all_126k_no_source.jsonl"]
target_path = "/content/LLaMA-Factory/data/"

# 执行拷贝操作
copy_files(input_path, file_names, target_path)


File 'Chinese_erotic.jsonl' copied to /content/LLaMA-Factory/data/
File 'Chinese_NovelWriting.jsonl' copied to /content/LLaMA-Factory/data/
File 'merge_all_126k_no_source.jsonl' copied to /content/LLaMA-Factory/data/


请为我实现一段python代码，对于/content/LLaMA-Factory/data/dataset_info.json 备份到dataset_info_back.json

In [10]:
import shutil
import os

original_file = '/content/LLaMA-Factory/data/dataset_info.json'
backup_file = '/content/LLaMA-Factory/data/dataset_info_back.json'

# Make a copy of the original file
shutil.copy(original_file, backup_file)

# Verify copy succeeded
if os.path.exists(backup_file):
    print(f'Backup of {original_file} to {backup_file} succeeded!')
else:
    print(f'Backup of {original_file} to {backup_file} failed!')

Backup of /content/LLaMA-Factory/data/dataset_info.json to /content/LLaMA-Factory/data/dataset_info_back.json succeeded!


In [15]:
import json

new_file = '/content/LLaMA-Factory/data/dataset_info.json'

with open(new_file, 'r') as f:
  data = json.load(f)

file_names = ["Chinese_erotic.jsonl", "Chinese_NovelWriting.jsonl", "merge_all_126k_no_source.jsonl"]

for i, filename in enumerate(file_names):
    data[f'haruhi_zero_{i}'] = {
    "file_name": filename,
    "formatting":"sharegpt",
    "columns": {
        "messages": "conversations",
        "system": "system",
    },
    "tags": {
        "role_tag": "from",
        "content_tag": "value",
        "user_tag": "human",
        "assistant_tag":"gpt",
    }
    }

with open(new_file, 'w') as f:
  json.dump(data, f, indent=2)

print('Data copied and new info appended to', new_file)

Data copied and new info appended to /content/LLaMA-Factory/data/dataset_info.json


In [12]:
!pip install -q tiktoken transformers_stream_generator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [13]:
!pip install -q datasets

In [ ]:
! CUDA_VISIBLE_DEVICES=0 python src/train_bash.py \
    --stage sft \
    --model_name_or_path  Qwen/Qwen-1_8B-Chat\
    --do_train True\
    --dataset haruhi_zero_0,haruhi_zero_1,haruhi_zero_2 \
    --template qwen \
    --finetuning_type lora \
    --lora_target c_attn \
    --lora_rank 16 \
    --lora_dropout 0.05 \
    --output_dir qwen_1_8-finetuned \
    --overwrite_output_dir \
    --overwrite_cache \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 1 \
    --lr_scheduler_type cosine \
    --logging_steps 100 \
    --save_steps 1000 \
    --learning_rate 2e-4 \
    --num_train_epochs 3.0 \
    --plot_loss \
    --fp16 True

2024-01-25 07:41:50.803083: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-25 07:41:50.803141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-25 07:41:50.804414: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-25 07:41:51.935026: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
01/25/2024 07:41:53 - WARNING - llmtuner.hparams.parser - `ddp_find_unused_parameters` needs to be set as False for LoRA in DDP training.
[INFO|training_args.py:1828] 2024-01-25 07:41:53,832 >> PyTorch: setting up devices
/usr/local/lib/pyth